In [1]:
# Import VTK library
from vtk import *

In [2]:
# Read in a dataset using the vtkXMLImageDataReader
reader = vtkXMLImageDataReader()
reader.SetFileName('Data/Isabel_2D.vti')
reader.Update()
data = reader.GetOutput()

In [3]:

# Get the number of cells and number of points in the dataset
numOfCells = data.GetNumberOfCells()
numOfPoints = data.GetNumberOfPoints()

In [4]:
# Get the dimensions and scalar dimensions of the dataset
dim = data.GetDimensions()
scalar_dim = data.GetDataDimension()


In [5]:
# Get the point data and pressure array
pdata = data.GetPointData()
pressureArr = pdata.GetArray('Pressure')

# Get the range of pressure values 
pressureRange = pressureArr.GetRange()

# Calculate the average pressure value
Sum = 0
for i in __builtins__.range(numOfPoints):
    Sum += pressureArr.GetValue(i)
avgPressure = Sum/numOfPoints


In [6]:
# Extracted cell with specified cellId
cellId = 0
vtkCell = data.GetCell(cellId)

# Getting Point Ids
pointIds = vtkCell.GetPointIds()

# Get the indices, coordinates, and pressure values of the points in the cell
indices = []
coordinates = []
pressureValues = []
for i in __builtins__.range(pointIds.GetNumberOfIds()):
    indices.append(pointIds.GetId(i))
    coordinates.append(data.GetPoint(vtkCell.GetPointId(i)))
    pressureValues.append(pressureArr.GetTuple1(indices[i]))

In [7]:
# Calculate the center point of the cell
center_x = 0
center_y = 0
center_z = 25
for i in __builtins__.range(pointIds.GetNumberOfIds()):
    center_x += coordinates[i][0]
    center_y += coordinates[i][1]
center_x /= 4
center_y /= 4
cell_center = (center_x, center_y, center_z)

In [8]:
# Calculate the average pressure value of the points in the cell
sum = 0
for i in pressureValues:
    sum += i
avgValues = sum/len(pressureValues)

In [9]:
# Print the number of cells, dimensions, and number of points

print("Number of cells:", numOfCells)
print("\nNumber of points:", numOfPoints)
print("\nThe dimensions of the dataset are: ", dim)
print("The dimensionality of the dataset : ", scalar_dim)

# Print range of pressure values present in dataset and average of those pressure values

print("\nRange of Pressure Values present in Dataset: ", pressureRange)
print("\nThe Average Pressure value of the entire dataset:", avgPressure)

# Print indices of Four corner vertice of cell, coordinates of each vertex and cell center
print("\nExtracted Cell with Cell Id: {0}".format(cellId))
print("\nIndices of 4 corner vertex:")
for i in indices:
    print(i,end=" ")
print()
print("\n3D Coordinate of each vertex:")
for i in coordinates:
    print(i)
print("\nCente of the Cell:",cell_center)

# Pressure values of 4 vertices of cell and their average
print("\nPressure Values of 4 vertices:\n",pressureValues)
print("\nAverage of Pressure values of vertices",avgValues)


Number of cells: 62001

Number of points: 62500

The dimensions of the dataset are:  (250, 250, 1)
The dimensionality of the dataset :  2

Range of Pressure Values present in Dataset:  (-1434.8590087890625, 630.5694580078125)

The Average Pressure value of the entire dataset: 240.77722069091325

Extracted Cell with Cell Id: 0

Indices of 4 corner vertex:
0 1 250 251 

3D Coordinate of each vertex:
(0.0, 0.0, 25.0)
(1.0, 0.0, 25.0)
(0.0, 1.0, 25.0)
(1.0, 1.0, 25.0)

Cente of the Cell: (0.5, 0.5, 25)

Pressure Values of 4 vertices:
 [477.527587890625, 474.79827880859375, 478.0115661621094, 467.60699462890625]

Average of Pressure values of vertices 474.4861068725586


In [10]:
# Create a vtkPolyData object
polyData = vtkPolyData()
points = vtkPoints()

In [11]:

# Add the coordinates of the points in the cell to the vtkPolydata object
for i in __builtins__.range(4):
    points.InsertNextPoint(coordinates[i])
polyData.SetPoints(points)

In [12]:
# Create an array of colors
colors = vtkUnsignedCharArray()
colors.SetNumberOfComponents(3)
colors.SetName("Colors")

#Assigning Different Color to each vertex
colors.InsertNextTuple3(255, 0, 0) # red
colors.InsertNextTuple3(0, 255, 0) # green
colors.InsertNextTuple3(0, 0, 255) # blue
colors.InsertNextTuple3(255, 255, 0) # yellow
polyData.GetPointData().SetScalars(colors)

0

In [13]:
# Create a glyph filter
glyphFilter = vtkVertexGlyphFilter()
glyphFilter.SetInputData(polyData)
glyphFilter.Update()


# Create a mapper
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(glyphFilter.GetOutputPort())


# Create an actor
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetPointSize(25)

In [14]:

# Create a renderer
renderer = vtkRenderer()

renderer.AddActor(actor)

# Create a render window
window = vtkRenderWindow()
window.SetSize(800,800)
window.AddRenderer(renderer)

# Create a render window interactor
interactor = vtkRenderWindowInteractor()
interactor.SetRenderWindow(window)

# Render the window
window.Render()
interactor.Initialize()
interactor.Start()